In [10]:
import requests
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine, text
from sqlalchemy.types import Integer, String, Date, Time

response = requests.get("https://api.jolpi.ca/ergast/2024/f1/2024/5/sprint/")
sprint_data = response.json()

In [11]:
round_number = sprint_data.get('MRData', {}).get('RaceTable', {}).get('round', None)
race_name = sprint_data.get('MRData', {}).get('RaceTable', {}).get('Races', [{}])[0].get('raceName', None)
race_year = sprint_data.get('MRData', {}).get('RaceTable', {}).get('Races', [{}])[0].get('season', None)
race_date = sprint_data.get('MRData', {}).get('RaceTable', {}).get('Races', [{}])[0].get('date', None)

In [12]:
sprint_data = sprint_data.get('MRData', {}).get('RaceTable', {}).get('Races', [{}])[0].get('SprintResults', None)
df = pd.DataFrame(sprint_data)

In [13]:
df['driver_id'] = df['Driver'].apply(lambda x: x['driverId'])
df['constructor_id'] = df['Constructor'].apply(lambda x: x['constructorId'])
df['driver'] = df['Driver'].apply(lambda x: f"{x['givenName']} {x['familyName']}")
df['constructor'] = df['Constructor'].apply(lambda x: x['name'])
df['time'] = df['Time'].apply(lambda x: x['time'] if isinstance(x, dict) else None)

In [14]:
df.insert(0, 'race_name', race_name)
df.insert(1, 'race_date', race_date)
df.insert(2, 'race_year', race_year)
df.insert(3, 'round_number', round_number)

In [15]:
df = df[['race_name', 'race_date', 'race_year', 'round_number', 
         'position', 'points', 'driver', 'driver_id', 
         'constructor', 'constructor_id', 'grid', 'status',]]

In [24]:
# PostgreSQL connection details
user = "f1_user"
password = "drivetosurvive"
host = "localhost"
port = "5433"
db = "drive_to_survive"

In [25]:
# Define SQL to drop and recreate the table
create_table_sql = """
DROP TABLE IF EXISTS sprint_2025;
CREATE TABLE sprint_2025 (
    race_name VARCHAR(255),
    race_date DATE,
    race_year INTEGER,
    round_number INTEGER,
    position INTEGER,
    points INTEGER,
    driver VARCHAR(255),
    driver_id VARCHAR(255),
    constructor VARCHAR(255),
    constructor_id VARCHAR(255),
    grid INTEGER,
    status VARCHAR(255)
);
"""

In [26]:
# Create SQLAlchemy engine
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{db}')

In [27]:
# Execute the table creation SQL
with engine.connect() as conn:
    conn.execute(text(create_table_sql))
    conn.commit()

In [23]:
dtype_mapping = {
    "race_name": String(255),
    "race_date": Date,
    "race_year": Integer,
    "round_number": Integer,
    "position": Integer,
    "points": Integer,
    "driver": String(255),
    "driver_id": String(255),
    "constructor": String(255),
    "constructor_id": String(255),
    "grid": Integer,
    "status": String(255),
}

df.to_sql('sprint_2025', engine, if_exists='replace', index=False, dtype=dtype_mapping)

20